# End-to-end training in CPU environment with PyTorch

In [1]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.file_list = [f for f in os.listdir(data_dir) if f.startswith("part_")]
        self.labels = np.load(os.path.join(data_dir, "labels.npy"), allow_pickle = True).item()
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        image = np.load(file_path)
        label = self.labels[self.file_list[idx]]
        return image, label

data_dir = "data/train_small_npy"

dataset = CustomDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=48, shuffle=True, num_workers=8)

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 125 * 125, 200)
        self.fc2 = nn.Linear(200, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 32 * 125 * 125)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Model()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [4]:
num_epochs = 10 # Originally set to 20, interrupted after 10

custom_transform = transforms.Compose([
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(90),
    transforms.RandomCrop((500, 500))
])

start_time = time.time()
for epoch in range(num_epochs):
    epoch_start = time.time()
    for images, labels in dataloader:
        
        transformed_images = torch.stack([custom_transform(image)/255.0 for image in images])
        
        outputs = model(transformed_images)
        labels = labels.view(-1, 1).float()
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    print(f"Epoch {epoch+1} done in {epoch_time} seconds.")
end_time = time.time()

total_time = end_time - start_time

print(f"Total time taken: {total_time} seconds")
print(f"Total time per epoch: {total_time/num_epochs} seconds")

Epoch 1 done in 456.24435091018677 seconds.
Epoch 2 done in 368.70431685447693 seconds.
Epoch 3 done in 382.88734769821167 seconds.
Epoch 4 done in 380.9022660255432 seconds.
Epoch 5 done in 386.09743213653564 seconds.
Epoch 6 done in 385.7481861114502 seconds.
Epoch 7 done in 373.4353709220886 seconds.
Epoch 8 done in 390.1825249195099 seconds.
Epoch 9 done in 384.55529618263245 seconds.
Epoch 10 done in 354.8102881908417 seconds.


KeyboardInterrupt: 